In [1]:
# ! pip install --user librosa

In [2]:
from pathlib import Path
from scipy.io import wavfile
import scipy.signal
import pandas as pd
from tqdm.auto import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
import librosa

## Read data

In [3]:
intermediate_folder = Path('..') / 'data' / 'intermediate'

In [4]:
X_train = np.load(intermediate_folder / 'train_main_1_sec_audio.npy').astype(float)
X_train.shape

(33566, 16000)

In [5]:
X_train

array([[-5.00e+00, -4.00e+00, -4.00e+00, ..., -9.00e+00, -7.00e+00,
        -1.50e+01],
       [-1.30e+02, -1.35e+02, -1.31e+02, ..., -1.54e+02, -1.90e+02,
        -2.24e+02],
       [ 1.50e+01,  2.90e+01,  1.50e+01, ...,  9.72e+02,  1.09e+03,
         9.34e+02],
       ...,
       [ 1.00e+00,  0.00e+00,  5.00e+00, ...,  1.10e+01,  1.00e+01,
         9.00e+00],
       [ 1.00e+00, -6.00e+00, -6.00e+00, ..., -1.20e+01, -1.30e+01,
        -5.00e+00],
       [-3.20e+01, -2.80e+01, -3.90e+01, ..., -2.00e+01, -4.20e+01,
        -2.80e+01]])

In [6]:
X_train.mean(axis=1).shape

(33566,)

In [7]:
X_train.mean(axis=1)

array([-4.78375000e-01, -9.63329375e+01,  1.93625000e-01, ...,
        2.33806250e+00,  6.75387500e+00,  7.83125000e-02])

In [8]:
X_val = np.load(intermediate_folder / 'val_main_1_sec_audio.npy').astype(float)
X_val.shape

(4619, 16000)

In [9]:
X_test = np.load(intermediate_folder / 'test_main_1_sec_audio.npy').astype(float)
X_test.shape

(4689, 16000)

In [10]:
SAMPLE_RATE = 16000

## Convert and Save librosa

In [11]:
# Compute STFT
stft = librosa.stft(X_train[0])
print(f'Sample shape: {X_train[0].shape}, initial stft {stft.shape=}, {stft.size=}')

# To ensure all the values are real, get magnitude
stft_magnitude, stft_phase = librosa.magphase(stft)
print(f'Before transpose: {stft_magnitude.shape=}')

# Transpose the result so the rows represent time
stft_magnitude = stft_magnitude.T
print(f'After transpose: {stft_magnitude.shape=}')

# To use this data with a Conv2D layer, it needs to have 4 dimensions: (num_samples, rows, columns, channels)
# If we're only using the magnitude (a common choice), we add an extra dimension for channels
stft_data = np.expand_dims(stft_magnitude, axis=-1)
print(f'After adding additional dimension {stft_data.shape=}')

Sample shape: (16000,), initial stft stft.shape=(1025, 32), stft.size=32800
Before transpose: stft_magnitude.shape=(1025, 32)
After transpose: stft_magnitude.shape=(32, 1025)
After adding additional dimension stft_data.shape=(32, 1025, 1)


In [12]:
def get_stft_librosa(raw_audio):
    # Compute STFT
    stft_magnitude, stft_phase = librosa.magphase(librosa.stft(raw_audio))
    return stft_magnitude

In [13]:
stft_train_list = []

for i in tqdm(range(len(X_train))):
    stft_train_list.append(get_stft_librosa(X_train[i]))

X_train_stft_np = np.stack(stft_train_list)
del stft_train_list
X_train_stft_np.shape

  0%|          | 0/33566 [00:00<?, ?it/s]

(33566, 1025, 32)

In [14]:
stft_val_list = []

for i in tqdm(range(len(X_val))):
    stft_val_list.append(get_stft_librosa(X_val[i]))

X_val_stft_np = np.stack(stft_val_list)
del stft_val_list
X_val_stft_np.shape

  0%|          | 0/4619 [00:00<?, ?it/s]

(4619, 1025, 32)

In [15]:
stft_test_list = []

for i in tqdm(range(len(X_test))):
    stft_test_list.append(get_stft_librosa(X_test[i]))

X_test_stft_np = np.stack(stft_test_list)
del stft_test_list
X_test_stft_np.shape

  0%|          | 0/4689 [00:00<?, ?it/s]

(4689, 1025, 32)

In [16]:
np.save(intermediate_folder / 'train_main_1_sec_audio_stft_librosa.npy', X_train_stft_np)

In [17]:
np.save(intermediate_folder / 'val_main_1_sec_audio_stft_librosa.npy', X_val_stft_np)

In [18]:
np.save(intermediate_folder / 'test_main_1_sec_audio_stft_librosa.npy', X_test_stft_np)

## Convert and Save Scipy

In [19]:
# Compute the STFT
frequencies, times, Zxx = scipy.signal.stft(X_train[0], fs=SAMPLE_RATE) # , nperseg=128, noverlap=56)

# Note: nperseg (number of samples per segment) and noverlap (number of samples to overlap) 
# can be adjusted according to your needs

# Compute the magnitude (absolute value) of the STFT complex numbers
Zxx = np.abs(Zxx)
frequencies.shape, times.shape, Zxx.shape


# # Plot the spectrogram
# plt.pcolormesh(times, frequencies, 10 * np.log10(Zxx), shading='gouraud')
# plt.title('Spectrogram')
# plt.ylabel('Frequency [Hz]')
# plt.xlabel('Time [sec]')
# plt.show()

((129,), (126,), (129, 126))

In [20]:
def get_stft_scipy(raw_audio):
    frequencies, times, Zxx = scipy.signal.stft(raw_audio, fs=SAMPLE_RATE) # , nperseg=128, noverlap=56)
    return np.abs(Zxx)

In [21]:
stft_train_list = []

for i in tqdm(range(len(X_train))):
    stft_train_list.append(get_stft_scipy(X_train[i]))

X_train_stft_np = np.stack(stft_train_list)
del stft_train_list
X_train_stft_np.shape

  0%|          | 0/33566 [00:00<?, ?it/s]

(33566, 129, 126)

In [22]:
stft_val_list = []

for i in tqdm(range(len(X_val))):
    stft_val_list.append(get_stft_scipy(X_val[i]))

X_val_stft_np = np.stack(stft_val_list)
del stft_val_list
X_val_stft_np.shape

  0%|          | 0/4619 [00:00<?, ?it/s]

(4619, 129, 126)

In [23]:
stft_test_list = []

for i in tqdm(range(len(X_test))):
    stft_test_list.append(get_stft_scipy(X_test[i]))

X_test_stft_np = np.stack(stft_test_list)
del stft_test_list
X_test_stft_np.shape

  0%|          | 0/4689 [00:00<?, ?it/s]

(4689, 129, 126)

In [24]:
np.save(intermediate_folder / 'train_main_1_sec_audio_stft_scipy.npy', X_train_stft_np)

In [25]:
np.save(intermediate_folder / 'val_main_1_sec_audio_stft_scipy.npy', X_val_stft_np)

In [26]:
np.save(intermediate_folder / 'test_main_1_sec_audio_stft_scipy.npy', X_test_stft_np)